In [ ]:
%load_ext autoreload
%autoreload 2
import waffles
import numpy as np
import json
import shutil 
from tqdm import tqdm

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.BasicWfAna import BasicWfAna
from waffles.data_classes.IPDict import IPDict
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.utils.baseline.baseline import SBaseline
from waffles.np02_utils.AutoMap import generate_ChannelMap, dict_uniqch_to_module, dict_module_to_uniqch
from waffles.np02_utils.PlotUtils import np02_gen_grids, plot_grid, plot_detectors, genhist, fithist, runBasicWfAnaNP02
from waffles.np02_utils.PlotUtils import plot_average_spe, matplotlib_plot_WaveformSetGrid

In [ ]:
dettype = "membrane"
# dettype = "cathode"




## Only change if necessary
datadir = f"/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/commissioning/"
det = "VD_Cathode_PDS" if dettype == "cathode" else "VD_Membrane_PDS"
endpoint = 106 if dettype == "cathode" else 107

# Way to low... keep scrollng
dletter = dettype.upper()[0] # C or M...
group1 = [ f"{dletter}{detnum}({chnum})" for detnum in range(1, 3) for chnum in range(1,3) ]
group2 = [ f"{dletter}{detnum}({chnum})" for detnum in range(3, 5) for chnum in range(1,3) ]
group3 = [ f"{dletter}{detnum}({chnum})" for detnum in range(5, 7) for chnum in range(1,3) ]
group4 = [ f"{dletter}{detnum}({chnum})" for detnum in range(7, 9) for chnum in range(1,3) ]
groupLow = group1+group2
groupHig = group3+group4
groupall = group1+group2+group3+group4

In [ ]:
from glob import glob
import copy
def open_processed(run, dettype, output_dir, channels = None, endpoints=None, nwaveforms=None, mergefiles = False, verbose=True):
    """
    Open the processed waveform set for a given run and detector type.
    """
    try: 
        wfset = load_structured_waveformset(
            f"{output_dir}/processed/run{run:0d}_{dettype}/processed_merged_run{run:06d}_structured_{dettype}.hdf5",
            max_to_load=nwaveforms,
            channels_filter=channels,
            endpoint_filter=endpoints
        )
    except:
        files = glob(f"{output_dir}/processed/run{run:06d}_{dettype}/processed_*_run{run:06d}_*_{dettype}.hdf5")
        if verbose:
            print("List of files found:")
            print(files)
        if not mergefiles or len(files)==1:
            files = files[0]
            wfset = load_structured_waveformset(files, max_to_load=nwaveforms, channels_filter=channels, endpoint_filter=endpoints, verbose=verbose)
        else: 
            wfset = load_structured_waveformset(files[0], max_to_load=nwaveforms, channels_filter= channels, endpoint_filter=endpoints, verbose=verbose)
            for f in files[1:]:
                tmpwf = load_structured_waveformset(f, max_to_load=nwaveforms, verbose=False)
                wfset.merge(copy.deepcopy(tmpwf))
    return wfset

In [ ]:
run_to_module = {

    "membrane" : {
        41230 : ["M3(1)", "M3(2)", "M4(1)", "M4(2)"],
        41233 : ["M5(1)", "M5(2)"],
        41234 : ["M6(1)", "M6(2)"],
        41235 : ["M7(1)", "M7(2)"],
        41236 : ["M8(1)", "M8(2)"],
    },

    "cathode" : {
        41223  : ["C1(1)", "C1(2)" ],
        41224  : ["C2(1)", "C2(2)"],
        41225  : ["C3(1)", "C3(2)"],
        41226  : ["C4(1)", "C4(2)"],
        41227  : ["C5(1)", "C5(2)"],
        41228  : ["C6(1)"],
        41229  : ["C6(2)"],
        41230  : ["C7(1)", "C7(2)"],
        41231  : ["C8(1)", "C8(2)"],
    }

}
run_to_module = run_to_module[dettype]

run_to_unich = { r: [ dict_module_to_uniqch[m].channel for m in modules ] for r, modules in run_to_module.items() }
channels = [ x for v in run_to_unich.values() for x in v]

In [ ]:
import copy
import time
def select_channels(waveform: Waveform, channels: list) -> bool:
    if waveform.channel not in channels:
        return False
    if np.any(waveform.adcs[0:500] > 16000):
        return False
    if np.any(waveform.adcs[0:500] < 100 ):
        return False
    return True
    
def create_wfset(run_to_unich, endpoint):

    nwaveforms = 20000
    wfset_full = None
    for run, channels in run_to_unich.items():
        wfset = open_processed(run, dettype, datadir, channels, [endpoint], nwaveforms=nwaveforms, verbose=True)
        # wfset = WaveformSet.from_filtered_WaveformSet(wfset, select_channels, channels)
        if wfset_full is None:
            wfset_full = copy.deepcopy(wfset)
        else:
            wfset_full.merge(copy.deepcopy(wfset))
        print(f"Loaded run {run}")
    return wfset_full
    
start = time.time()
wfset_full = create_wfset(run_to_unich, endpoint)
end = time.time()
print(end - start)

In [ ]:
def filter_bad_waveforms(waveform: Waveform) -> bool:
    # if np.any(waveform.adcs[0:500] > 16000):
    #     return False
    # if np.any(waveform.adcs[0:500] < 100 ):
    #     return False
    return True
wfset_full = WaveformSet.from_filtered_WaveformSet(wfset_full, filter_bad_waveforms, show_progress=True)
print(len(wfset_full.waveforms))

In [ ]:
wfset_full

In [ ]:
from pathlib import Path
from datetime import datetime 

histargs = dict(    
    # doprocess = False,
    dofit = True,
    show_progress = True,
    fitmultigauss = True,
    # verbosemultigauss = True,
    # configyaml = "ch_snr_parameters.yaml",
)
setupargs=dict(
# default 1300x1300
    width=1300,
    height=650,
    showplots=True
)
htmlname = None

# timenow = datetime.now().strftime('%Y%m%d_%H%M%S') 
# htmlname = Path(f"./calibration_{dettype}_{timenow}.html")

# # Any detector can be processed alone if all were processed already
# detector=group1
# plot_detectors(wfset_full, detector, plot_function=fithist, html=None, wf_func=histargs)
  
# detector=["M8(1)", "M8(2)"]
# plot_detectors(wfset_full, detector, plot_function=fithist, html=None, wf_func=histargs)
detector=groupLow
plot_detectors(wfset_full, detector, plot_function=fithist, html=htmlname, wf_func=histargs, **setupargs)
detector=groupHig
plot_detectors(wfset_full, detector, plot_function=fithist, html=htmlname, wf_func=histargs, **setupargs)

In [ ]:
argsheat = dict(
    mode="heatmap",
    analysis_label="std",
    adc_range_above_baseline=200,
    adc_range_below_baseline=-50,
    adc_bins=125,
    time_bins=wfset_full.points_per_wf//2,
    filtering=4,
    share_y_scale=False,
    share_x_scale=True,
    wfs_per_axes=5000,
    zlog=True,
    width=1300,
    higth=650
)
detector=groupLow
plot_detectors(wfset_full, detector, **argsheat)

detector=groupHig
plot_detectors(wfset_full, detector, **argsheat)


In [ ]:
func_params = dict(
    show_progress=False,
    pretrigger=200,
    posttrigger=400,
    threshold=80
)
detector = groupLow+groupHig
matplotlib_plot_WaveformSetGrid(wfset_full, plot_function=plot_average_spe, detector=detector, func_params=func_params)
# detector = groupHig
# matplotlib_plot_WaveformSetGrid(wfset_full, plot_function=plot_average_spe, detector=detector, func_params=func_params)

In [ ]:
wfset_ch = ChannelWsGrid.clusterize_waveform_set(wfset_full)

In [ ]:
wfs = wfset_ch[endpoint][32]

In [ ]:
list_of_bad_apples = []
for i, w in enumerate(wfs.waveforms):
    v = w.adcs - w.analyses['std'].result['baseline']
    # if abs(w.analyses['std'].result['integral']) < 200:
    list_of_bad_apples.append(v)
        # print(i)

In [ ]:
import matplotlib.pyplot as plt
a+=1
plt.plot(list_of_bad_apples[a])